In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import disk, opening, closing
import glob
import pathlib

In [2]:
rpi = 'h2r3'

# Read input image and resize them
path_to_folder = '../live_bees/' + rpi + '/random_images_25/*.jpg'
folder_in = sorted(glob.glob(path_to_folder))
names = [pathlib.Path(x).stem for x in folder_in]

folder_out = 'resized_images/' + rpi + '/'
n = 0
for img_path in folder_in:
    cv2.imwrite(folder_out + str(names[n]) + '.jpg', cv2.resize(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2GRAY), (0,0), fx=0.5, fy=0.5));
    n += 1

resized_imgs_path = sorted(glob.glob(folder_out + '*.jpg'))

(1296, 2304, 3)


# Run this cell only if you want to batch process using ilastik

In [122]:
resized_imgs_path = sorted(glob.glob(folder_out + '*.jpg'))

ILASTIK_PATH = '/Users/fabianprofeta/Desktop/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/run_ilastik.sh'
MODEL='trained_model.ilp'
IMG = ''
OUTPUT_FORMAT = 'bmp'
OUTPUT_FOLDER = './segmentation/'+ rpi + '/{nickname}_mask.bmp'
for img_path in resized_imgs_path:
    IMG += img_path + ' '
# we call ilastik as headless operation on the pc
!$ILASTIK_PATH  --headless \
                --project=$MODEL \
                --export_source='simple segmentation' \
                --output_format=$OUTPUT_FORMAT \
                --output_filename_format=$OUTPUT_FOLDER \
                $IMG

INFO ilastik.app: Using tiktorch executable: ['/Users/fabianprofeta/Desktop/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/bin/python', '-m', 'tiktorch.server']
INFO ilastik.app: config file location: <none>
INFO ilastik.app: Starting ilastik from "/Users/fabianprofeta/Desktop/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7".
Starting ilastik from "/Users/fabianprofeta/Desktop/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/lib/python3.7".
WARNING 2024-05-30 15:36:47,303 opConservationTracking 46378 4656752128 Could not find any ILP solver
WARNING 2024-05-30 15:36:47,342 opStructuredTracking 46378 4656752128 Could not find any ILP solver
WARNING 2024-05-30 15:36:47,345 structuredTrackingWorkflow 46378 4656752128 Could not find any learning solver. Tracking will use flow-based solver (DPCT). Learning for tracking will be disabled!
INFO ilastik.shell.projectManager: Opening Project: trained_model.ilp
INFO ilastik.workflows.pixelClassification.pixelClassifica

# Post processessing

In [4]:
path_to_segmented = sorted(glob.glob('segmentation/' + rpi + '/*.bmp'))
path_to_processed = '../a_processed_images/ilastik/' + rpi + '/'
segmented_imgs = []
for path in path_to_segmented:
    segmented_imgs.append(cv2.imread(path, cv2.IMREAD_GRAYSCALE))
n = 0
processed_imgs = []
for mask in segmented_imgs:
    mask_open = opening(mask, disk(5))
    mask_closed = closing(mask_open, disk(5))
    processed_imgs.append(mask_closed)
    cv2.imwrite(path_to_processed + names[n] + '.bmp', mask_closed)
    n += 1